In [39]:
title = 'results_1613085753.6759956'
columns = ['$TF_{us}$ + $TF_{un}$',
           '$TF_{us}$',
           '$TF_{un}$',
           '$Q_{c}$',
           '$Q_{p}$',
           '$Q_{c}$ + $Q_{p}$',
           '$Q_{l}$',
           '$Q_{d}$',
           '$Q_{l}$ + $Q_{d}$',
           '$Q_{l}$ + $Q_{s}$ + $Q_{c}$ + $Q_{p}$',
           '$TF_{bs}$ + $TF_{bn}$',
           '$TF_{bs}$',
           '$TF_{bn}$',
           '$TF_{us}$ + $TF_{un}$ + $TF_{bs}$ + $TF_{bn}$',
           '$TF_{us}$ + $TF_{un}$ + $Q_{l}$ + $Q_{s}$ + $Q_{c}$ + $Q_{p}$',]
with open(f'./data/knn/reports/{title}.txt', 'r') as fp:
    lines = fp.readlines()
    for i in range(len(columns)):
        string_to_print = '{%s} &' % columns[i]
        print(string_to_print, lines[i], end='')


{$TF_{us}$ + $TF_{un}$} & {0.99766}&{0.91856}&{0.91929}&{0.99765}&{0.95683}&{0.95643} \\
{$TF_{us}$} & {0.97531}&{0.86193}&{0.85750}&{0.97608}&{0.91256}&{0.91542} \\
{$TF_{un}$} & {0.83160}&{0.94887}&{0.96143}&{0.78588}&{0.89175}&{0.85958} \\
{$Q_{c}$} & {0.67145}&{0.66840}&{0.72393}&{0.61059}&{0.69653}&{0.63792} \\
{$Q_{p}$} & {0.57922}&{0.57195}&{0.69643}&{0.44471}&{0.63241}&{0.50030} \\
{$Q_{c}$ + $Q_{p}$} & {0.82822}&{0.82452}&{0.84321}&{0.80784}&{0.83557}&{0.81600} \\
{$Q_{l}$} & {0.58900}&{0.56886}&{0.65750}&{0.49608}&{0.62127}&{0.52982} \\
{$Q_{d}$} & {0.60660}&{0.56897}&{0.60929}&{0.56588}&{0.60769}&{0.56714} \\
{$Q_{l}$ + $Q_{d}$} & {0.74964}&{0.67527}&{0.66893}&{0.75490}&{0.70689}&{0.71279} \\
{$Q_{l}$ + $Q_{s}$ + $Q_{c}$ + $Q_{p}$} & {0.98333}&{0.98625}&{0.98750}&{0.98157}&{0.98539}&{0.98388} \\
{$TF_{bs}$ + $TF_{bn}$} & {0.94043}&{0.68708}&{0.60214}&{0.95804}&{0.73387}&{0.80015} \\
{$TF_{bs}$} & {0.87667}&{0.60851}&{0.45464}&{0.92980}&{0.59826}&{0.73552} \\
{$TF_{bn}$} & {0